# Stuying the harmonic expansions of functions

## Preliminaries

### Spherical coordinate system

A vector is written as $\mathbf{r}=\left(r,\varphi,\theta\right)^t$, with $r \in [0,\infty)$, $\varphi \in [0,2\pi)$ and $\theta \in [0,\pi]$, which in Cartesian coordinates is equivalent to $\mathbf{r}=r\left(\sin \theta \cos \varphi,\sin \theta \sin \varphi,\cos \theta\right)^t$. The unitary vectors of the spherical coordinate system can be written in Cartesian coordinates as
$$\begin{align*}
    &\widehat{\mathbf{e}}_r= (\sin \theta \cos \varphi , \sin \theta \sin \varphi , \cos \theta )^t, \\
    &\widehat{\mathbf{e}}_\theta=(\cos \theta \cos \varphi , \cos \theta \sin \varphi , -\sin \theta  )^t, \\
    &\widehat{\mathbf{e}}_{\varphi}=(-\sin \varphi , \cos \varphi , 0 )^t.
\end{align*}$$

Also, the gradient operator in spherical coordinates:
$$\begin{align*}
    \nabla f&=\frac{\partial f}{\partial r} \widehat{\mathbf{e}}_r+\frac{1}{r} \frac{\partial f}{\partial \theta} \widehat{\mathbf{e}}_\theta+\frac{1}{r \sin \theta} \frac{\partial f}{\partial \varphi} \widehat{\mathbf{e}}_{\varphi}
\end{align*}$$

### Real spherical harmonics

Real spherical harmonics of degree $l$ and order $m$ are defined using spherical coordinates:
$$\begin{align}
		 Y_{l,m}\left(\theta,\varphi\right) &:= \sqrt{ (2-\delta_{m,0}) \frac{\left(2l+1\right)\left(l-m\right)!}{4 \pi \left(l+m\right)!}} P_l^{m} \left(\cos\theta\right) \cos m \varphi , \mbox{ and}\\
		Y_{l,-m}\left(\theta,\varphi\right) &:= \sqrt{ (2-\delta_{m,0})\frac{\left(2l+1\right)\left(l-m\right)!}{4 \pi \left(l+m\right)!}} P_l^{m} \left(\cos\theta\right) \sin m \varphi ,
\end{align}$$
with $l\in \mathbb{N}_0$, $m\in \mathbb{Z}$ such that $0\leq m\leq l$. If $m=0$, then $\delta_{m,0}=1$, and it is zero otherwise. $P_l^m$ are the associated Legendre functions of degree $l$ and  order $m$ defined as:
$$P_{l}^m\left(x\right) := (-1)^m \left( 1- x^2\right)^{\frac{m}{2}} \frac{d^m}{dx^m}P_l(x), \quad \mbox{with} \quad P_{l}\left(x\right) := \frac{1}{2^ll!}\frac{d^l}{dx^l}(x^2-1)^l.$$

Here, the term $(-1)^m$ is the Condon-Shortley phase factor.

### Orthonormal basis

Spherical harmonics are dense in $C(\mathbb{S}^2)$, with $\mathbb{S}^2$ the surface of the unit sphere, and form a complete orthonormal system in $L^2(\mathbb{S}^2)$ with respect to the internal product defined by:
$$\left( \psi , \xi \right)_{L^2(\mathbb{S}^2)} = \int_{0}^{2\pi}\int_{0}^{\pi} \psi\left(\theta,\varphi\right) \xi\left(\theta,\varphi\right) \sin\left(\theta\right) d\theta d\varphi,$$
They also are orthogonal in $H^1(\mathbb{S}^2)$.

This means that any function in $L^2(\mathbb{S}^2)$ can be written as:

$$f(\theta,\varphi) = \sum_{l=0}^\infty \sum_{m=-l}^{m=l} f_{l,m} Y_{l,m}(\theta, \varphi)$$
with $f_{l,m}\in \mathbb{R}$.

### Comments about the code.

The module `biosspheres.miscella.harmonicex` give us an approximation of the $f_{l,m}$ coefficients for some functions. Aditionally, the package SHTOOLS has a function to approximate $f_{l,m}$ given the values of the function in some points in the surface of the sphere, see [SHTOOLS - spherical harmonic transforms](https://shtools.github.io/SHTOOLS/python-spherical-harmonic-transforms.html).

## Constant function

A function of the form $f(x) = c$ has only the first $f_{l,m}$ different from zero.

The following is the known dirichlet expansion for $f(x) = $ `cte`:

In [11]:
import numpy as np
import biosspheres.miscella.harmonicex as harmonicex

cte = 5.
big_l = 3
dirichlet_expansion = harmonicex.function_cte_dirichlet(big_l, cte, azimuthal=False)
dirichlet_expansion

array([17.72453851,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

The following is the approximated dirichlet expansion with the package SHTOOLS, from the values in the sphere.

In [12]:
import pyshtools
import biosspheres.miscella.auxindexes as auxindexes
import biosspheres.quadratures.sphere as quadratures

max_l = big_l
quantity_theta_points, quantity_phi_points, w, pre_vector = quadratures.gauss_legendre_trapezoidal_2d(max_l)
grid_analytic = np.ones((quantity_theta_points, quantity_phi_points)) * cte
coefficients = pyshtools.expand.SHExpandGLQ(grid_analytic, w, pre_vector[2, :, 0], norm=4, csphase=-1, lmax_calc=max_l)
dirichlet_expansion_2 = np.empty_like(dirichlet_expansion)

pesykus, p2_plus_p_plus_q, p2_plus_p_minus_q = auxindexes.pes_y_kus(max_l)
eles = np.arange(0, max_l + 1)
l_square_plus_l = eles * (eles + 1)

dirichlet_expansion_2[p2_plus_p_plus_q] = coefficients[0, pesykus[:, 0], pesykus[:, 1]]
dirichlet_expansion_2[p2_plus_p_minus_q] = coefficients[1, pesykus[:, 0], pesykus[:, 1]]
dirichlet_expansion_2[l_square_plus_l] = coefficients[0, eles, 0]

dirichlet_expansion_2

array([ 1.77245385e+01, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00,  1.06581410e-14,  0.00000000e+00,
        5.80672801e-16, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -2.76647309e-16])

The difference between the results is

In [13]:
np.linalg.norm(dirichlet_expansion - dirichlet_expansion_2)

1.2825629922959107e-14

## Linear function